In [ ]:
import xarray as xr
import param

import holoviews as hv
import hvplot.xarray
from holoviews.selection import link_selections

import panel as pn

import io
import os

In [ ]:
class ValueChanger(param.Parameterized):
    
    calculation_type = pn.widgets.RadioButtonGroup(options=['Absolute', 'Relatif', 'Percentage'], align='end')
    save = pn.widgets.FileDownload(label='Save', align='end', button_type='success')
    apply = pn.widgets.Button(name='Apply', align='end', button_type='primary')
    spinner = pn.widgets.Spinner(name='Replacement Value', value=0, align='end')
    attribute = param.String()
    file = param.Parameter()
    
    ds = param.Parameter()
    selection = link_selections.instance(unselected_alpha=0.4)
    
    app_pane = pn.Column()
    
    def __init__(self, **params):
        self.param.file.default = pn.widgets.FileInput()
        self.param.ds.default = xr.Dataset()
        self.param.attribute.default = pn.widgets.Select(max_width=200, align='start')
        super().__init__(**params)
        self.apply.on_click(self._apply_values)
        self.save.callback = self._save
        
    @pn.depends("file.value", watch=True)
    def _parse_file_input(self):
        value = self.file.value
        ds = xr.open_dataset(value)
        self.attribute.options = list(ds.keys())
        self.ds = ds
        
    def _set_values(self):
        hvds = hv.Dataset(self.ds.to_dataframe().reset_index())
        if self.calculation_type.value == 'Absolute':
            hvds.data[self.attribute.value].loc[hvds.select(self.selection.selection_expr).data.index] = self.spinner.value
        elif self.calculation_type.value == 'Relatif':
            hvds.data[self.attribute.value].loc[hvds.select(self.selection.selection_expr).data.index] += self.spinner.value
        elif self.calculation_type.value == 'Percentage':
            hvds.data[self.attribute.value].loc[hvds.select(self.selection.selection_expr).data.index] *=  (100 + self.spinner.value) / 100.
        self.ds[self.attribute.value] = list(self.ds.coords.keys()), hvds.data[self.attribute.value].values.reshape(*self.ds[self.attribute.value].shape)
        
    def _save(self):
        filename, extension = os.path.splitext(self.file.filename) 
        self.save.filename = filename + "_netcdf-editor" + extension
        return io.BytesIO(self.ds.to_netcdf())
    
    def _apply_values(self, event):
        self._set_values()
        self.selection.selection_expr = None
    
    @pn.depends('ds', watch=True)
    def get_plots(self):
        self.app_pane.clear()
        self.app_pane.extend([
            pn.Row(self.attribute, self.save),
            pn.Row(self.calculation_type, self.spinner, self.apply), 
            self.selection(self.ds.hvplot() + self.ds[self.attribute.value].hvplot.hist()).opts(hv.opts.Image(tools=['hover']), hv.opts.Histogram(tools=['hover']))
        ])
    
    def plot(self):
        return pn.Column(
            self.file,
            self.app_pane
        )

In [ ]:
vc = ValueChanger()
vc.plot().servable()